# Confiabilidade de viga mista

## Importando bibliotecas

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import import_ipynb
from math import sqrt

## Definindo funções

In [8]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    amostra: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [9]:
espessura_laje: float = 10 # cm 
desvpad_espessura_laje: float = 0.06 * espessura_laje # cm

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [10]:
limite_escoamento_perfil: float = 250 * 1.08 # MPa
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil # MPa

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [11]:
limite_escoamento_armadura: float = 500 * 1.08 # MPa
desvpad_escoamento_armadura: float = 0.08 * limite_escoamento_armadura #MPa

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [12]:
resistencia_concreto: float = 20 * 1.17 # MPa
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto # MPa

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [13]:
diametro_rebite: float = 1.59 # cm
desvpad_diametro_rebite: float = 0.01 * diametro_rebite # cm

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [14]:
resistencia_rebite: float = 415*1.06 # MPa
desvpad_resistencia_rebite: float = 0.06 * resistencia_rebite # MPa

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [15]:
carga_permanente_g1: float = 7.6 # kN/m
carga_permanente_g2: float = 5 # kN/m

variacao_carga: list = [3, 11]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9 # m
espacamento_laje: float = 2.8 # m

# Carga distrubuída do projeto
carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i] for i in range(len(carga_utilizacao_q2))]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * vao_laje ** 2) / 8 for i in range(len(carga_distribuida_projeto))]

altura_laje_ht: float = 45 #cm
coeficiente_ya1: float = 1.1
x: float = 10 # cm

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_laje_ht / 2) + espessuras_laje[j] - (x / 2)))) # cm²
    index_matriz += 1
    
print(areas_minimas) 


[[15.74210387773236, 13.400999979595694, 14.063733529549362, 14.127777384563608, 13.814191389809189, 12.588545198597142, 11.595014209942082, 14.720333206157818, 14.186461884111388, 15.450334774868221, 12.872835786718523, 12.74985297256389, 13.705309743487174, 14.247683178177711, 14.509644674222635, 16.614420250511188, 14.629052608742658, 15.911086137473935, 13.178888734439267, 13.502505557930807, 13.620269141805698, 13.391183167678696, 14.605606822171525, 16.68085246156753, 16.55665954696753, 15.17842062324648, 16.570232777007433, 13.703295789003771, 16.35237304241444, 13.515965796266459, 14.782168542611226, 16.82580345779609, 15.467715046728893, 13.27942569542034, 15.947369358072246, 14.346764671110115, 15.451329601081351, 15.65700099387634, 13.246688237884035, 16.581744736617388, 14.884916129530666, 13.809003453733952, 14.244022655092373, 13.679146133063986, 15.220603414544915, 13.991728643398345, 16.42788830499566, 13.754718594562597, 14.269838502708504, 14.045038977359566, 14.00929

## Flambagem da seção

In [16]:
# Classificação da seção quanto a flambagem

altura_perfil: float = 404 # mm
espessura_perfil: float = 7.6 # mm
modulo_elasticidade_perfil: float = 200000 # MPa

esbeltez_alma: float = altura_perfil / espessura_perfil

limites_flambagem_local: list = []

def calculo_limite_flambagem_local(coef: float, modulo_elasticidade_perfil: float, limite_escoamento_perfil: float) -> float:
    return coef * sqrt(modulo_elasticidade_perfil / limite_escoamento_perfil)

countador_compacto: int = 0
countador_semicompacto: int = 0

for i in range(len(limites_escoamento_perfil)):
    limite = calculo_limite_flambagem_local(3.76, modulo_elasticidade_perfil, limites_escoamento_perfil[i])
    limites_flambagem_local.append(limite)
    
    if esbeltez_alma <= limite:
        countador_compacto += 1
    elif esbeltez_alma > limite and esbeltez_alma < calculo_limite_flambagem_local(5.7, modulo_elasticidade_perfil, limites_escoamento_perfil[i]):
        countador_semicompacto += 1
    else:
        print('Erro')
        
print(f'Compactas: {countador_compacto}')
print(f'Semi-compactas: {countador_semicompacto}')
        

Compactas: 1000
Semi-compactas: 0
